In [7]:
import pandas as pd
import numpy as np
import shap 
import xgboost as xgb
from sklearn.multioutput import MultiOutputRegressor
from sklearn.model_selection import train_test_split
from tqdm.notebook import tqdm
import pickle 
from sklearn.metrics import r2_score, mean_squared_error, make_scorer
import os
import multiprocessing as mp
from tqdm.contrib.concurrent import process_map, thread_map
from sklearn.model_selection import cross_val_score

In [8]:
# Reading in full data files
gene_expression = pd.read_csv(('~/Zhang-Lab/Zhang Lab Data/Full data files/Geneexpression (full).tsv'), sep='\t', header=0)
tf_expression = pd.read_csv(('~/Zhang-Lab/Zhang Lab Data/Full data files/TF(full).tsv'), sep='\t', header=0)

In [3]:
# Split into training, testing and validation sets and into numpy arrays + combining dataframes
x = tf_expression
y = gene_expression

combined_data = pd.concat([x, y], axis=1)

# First split: 70% train and 30% temp (test + val)
x_train, x_temp, y_train, y_temp = train_test_split(
    x, y, test_size=0.3, random_state=42)

# Second split: split the temp set into 20% test and 10% val (which is 2/3 and 1/3 of temp)
x_test, x_val, y_test, y_val = train_test_split(
    x_temp, y_temp, test_size=1/3, random_state=42)


# For training set
x_train = x_train.to_numpy()
y_train = y_train.to_numpy()

# For validation set
x_val = x_val.to_numpy()
y_val = y_val.to_numpy()

# For testing set
x_test = x_test.to_numpy()
y_test = y_test.to_numpy()

In [4]:
# Load model in with pickle file 

with open('/home/christianl/Zhang-Lab/Zhang Lab Data/Saved models/Random Forest/RF_model.pkl', 'rb') as file:
    loaded = pickle.load(file)

/tmp/ipykernel_3422478/2140450883.py:4: UserWarning: [20:56:11] WARNING: /home/conda/feedstock_root/build_artifacts/xgboost-split_1763746907527/work/src/collective/../data/../common/error_msg.h:83: If you are loading a serialized model (like pickle in Python, RDS in R) or
configuration generated by an older version of XGBoost, please export the model by calling
`Booster.save_model` from that version first, then load it back in current version. See:

    https://xgboost.readthedocs.io/en/stable/tutorials/saving_model.html

for more details about differences between saving model and serializing.

  loaded = pickle.load(file)


In [ ]:
 # Evaluate model 

 #loaded = pickle.load(file)
 #Loaded object is a list of models. n_models: 16101
 #y_pred.shape: (3187, 16101)
 #model.n_features_in_: None
 #model.n_outputs_: None
 #x_test.shape: (3187, 1198)
 #y_test.shape: (3187, 16101)
 #Multi-output R^2 (uniform_average): 0.7601639389405871


# List of estimators (one per target gene) so build predictions matrix 

if isinstance(loaded, list):
    models = loaded
    print("Loaded object is a list of models. n_models:", len(models))
    y_pred = np.column_stack([m.predict(x_test) for m in models])  # (n_samples, n_genes)
else:
    model = loaded
    print("Loaded object type:", type(model))
    # If single-output model but y_test is multi-column, this will raise -- handled later
    y_pred = model.predict(x_test)

print("y_pred.shape:", y_pred.shape)

# diagnostics (safe access)
def safe_attr(obj, name):
    return getattr(obj, name, None) if not isinstance(obj, list) else None

print("model.n_features_in_:", safe_attr(loaded, "n_features_in_"))
print("model.n_outputs_:", safe_attr(loaded, "n_outputs_"))
print("x_test.shape:", x_test.shape)
print("y_test.shape:", y_test.shape)

# Continue with your R^2 / MSE logic expecting y_pred shape (n_samples, n_targets)
from sklearn.metrics import r2_score, mean_squared_error

# If y_pred is 1D, treat as single-output
if y_pred.ndim == 1 or (y_pred.ndim == 2 and y_pred.shape[1] == 1):
    if y_test.ndim == 2 and y_test.shape[1] > 1:
        raise ValueError(
            "Model predicts a single target but y_test contains multiple targets (genes).\n"
            "Select the trained target column before splitting, e.g.:\n"
            "  target = 'GENE_NAME'\n"
            "  y = gene_expression[target]\n"
            "  then redo train_test_split and evaluation."
        )
    y_true = y_test.ravel()
    print("R^2:", r2_score(y_true, y_pred))
    print("MSE:", mean_squared_error(y_true, y_pred))
else:
    print("Multi-output R^2 (uniform_average):", r2_score(y_test, y_pred, multioutput='uniform_average'))
    
    
first = models[0]
print("first estimator type:", type(first))
print("n_features_in_:", getattr(first, "n_features_in_", None))
print("example feature_importances_ (first 10):", getattr(first, "feature_importances_", None)[:10])


/tmp/ipykernel_2257987/2997833905.py:4: UserWarning: [19:18:09] WARNING: /home/conda/feedstock_root/build_artifacts/xgboost-split_1763746907527/work/src/collective/../data/../common/error_msg.h:83: If you are loading a serialized model (like pickle in Python, RDS in R) or
configuration generated by an older version of XGBoost, please export the model by calling
`Booster.save_model` from that version first, then load it back in current version. See:

    https://xgboost.readthedocs.io/en/stable/tutorials/saving_model.html

for more details about differences between saving model and serializing.

  loaded = pickle.load(file)


Loaded object is a list of models. n_models: 16101
y_pred.shape: (3187, 16101)
model.n_features_in_: None
model.n_outputs_: None
x_test.shape: (3187, 1198)
y_test.shape: (3187, 16101)
Multi-output R^2 (uniform_average): 0.7601639389405871


In [5]:
def objective(trial):
    n_estimators = trial.suggest_int("n_estimators", 100, 1000)
    max_depth = trial.suggest_int("max_depth", 3, 20)

    subsample = trial.suggest_float("subsample", 0.5, 1.0)
    colsample_bytree = trial.suggest_float("colsample_bytree", 0.5, 1.0)
    reg_alpha = trial.suggest_float("reg_alpha", 0.0, 5.0)
    reg_lambda = trial.suggest_float("reg_lambda", 0.0, 10.0)
    learning_rate = trial.suggest_float("learning_rate", 0.1, 1.0)

    # Use the RF regressor class; note the name
    model = xgb.XGBRFRegressor(
        n_estimators=n_estimators,
        max_depth=max_depth,
        subsample=subsample,
        colsample_bytree=colsample_bytree,
        reg_alpha=reg_alpha,
        reg_lambda=reg_lambda,
        learning_rate=learning_rate,
        n_jobs=-1,
    )

    multi_r2 = make_scorer(r2_score, multioutput="uniform_average")

    score = cross_val_score(
        model, x_train, y_train, cv=5, n_jobs=-1, scoring=multi_r2
    ).mean()

    return score

In [10]:
import optuna
from sklearn.model_selection import cross_val_score
from sklearn.metrics import r2_score, make_scorer
import xgboost as xgb

study = optuna.create_study(direction='maximize', sampler=optuna.samplers.RandomSampler(seed=42)) # Default is random Search

[I 2025-12-02 20:59:08,777] A new study created in memory with name: no-name-a616f325-38d2-4889-b1f9-def3675d9fed


In [11]:
from tqdm import tqdm

class TqdmCallback:
    def __init__(self, total_trials):
        self.pbar = tqdm(total=total_trials, desc="Optuna Optimization")
    
    def __call__(self, study, trial):
        self.pbar.update(1)
        self.pbar.set_postfix({"best_r2": f"{study.best_value:.4f}"})
    
    def __del__(self):
        self.pbar.close()

callback = TqdmCallback(total_trials=100)
study.optimize(objective, n_trials=100, callbacks=[callback])

Optuna Optimization:   0%|          | 0/100 [02:43<?, ?it/s]
[W 2025-12-02 21:00:00,375] Trial 0 failed with parameters: {'n_estimators': 437, 'max_depth': 20, 'subsample': 0.8659969709057025, 'colsample_bytree': 0.7993292420985183, 'reg_alpha': 0.7800932022121826, 'reg_lambda': 1.5599452033620265, 'learning_rate': 0.15227525095137953} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "/home/christianl/miniconda3/envs/remote_training/lib/python3.12/site-packages/optuna/study/_optimize.py", line 201, in _run_trial
    value_or_values = func(trial)
                      ^^^^^^^^^^^
  File "/tmp/ipykernel_3422478/3709679077.py", line 25, in objective
    score = cross_val_score(
            ^^^^^^^^^^^^^^^^
  File "/home/christianl/miniconda3/envs/remote_training/lib/python3.12/site-packages/sklearn/utils/_param_validation.py", line 218, in wrapper
    return func(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^
  File "/home/christianl/minico

KeyboardInterrupt: 